In [2]:
#packages
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.linalg import qr
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from scipy.linalg import lstsq

In [3]:
def fast_sketch_matrix(m, n_columns):
    row_indices = np.random.randint(0, m, size=n_columns)  # Pick one random row per column
    col_indices = np.arange(n_columns)  # Each column gets one entry
    values = np.random.choice([1, -1], size=n_columns)  # ±1 values

    # Directly construct the sparse matrix
    S = csr_matrix((values, (row_indices, col_indices)), shape=(m, n_columns))

    return S

In [4]:
def gaussian_sketch_matrix(m, n):
    sketch_matrix = np.random.randn(m, n)
    return sketch_matrix

In [5]:
def sketch_matrix(m, n_columns, non_zero_entries):
    #matrix with all zero entries
    S = np.zeros((m, n_columns))
    scaling_factor = 1 / np.sqrt(non_zero_entries)
    
    #loop through each columns to edit the non zero entries in
    for col in range(n_columns):
        # Randomly select position of non_zero entries
        nz_positions = np.random.choice(m, non_zero_entries, replace=False)
        
        # Randomly assign values of either 1 or -1 to these positions
        values = np.random.choice([1, -1], non_zero_entries)* scaling_factor
        
        # Assign the values to the selected positions in the column
        for idx, value in zip(nz_positions, values):
            S[idx, col] = value
    
    return S